# Connect4

#####
O código completo do projeto se encontra em:https://github.com/sofiabucci/AIProject

O objetivo do projeto foi desenvolver um programa capaz de jogar **Connect Four** contra humanos ou outros algoritmos, utilizando:

1. **Monte Carlo Tree Search (MCTS)** com **UCT**.
2. **Árvores de decisão** construídas com o algoritmo **ID3**.

#### 1. **Implementação do jogo Connect Four**

* Suportar três modos:
  * humano vs. humano
  * humano vs. computador
  * computador vs. computador (duelo entre MCTS e Decision Tree)

#### 2. **Monte Carlo Tree Search (MCTS)**
* Implementar o algoritmo MCTS com **Upper Confidence Bound for Trees (UCT)**.
* Avaliar diferentes quantidades de filhos selecionados por nó.

#### 3. **Árvores de Decisão com ID3**
* Criar uma árvore de decisão a partir de **dois conjuntos de dados**:
  1. **Dataset 1** (iris): fornecido na plataforma Moodle.
     * Requer discretização dos valores numéricos.
  2. **Dataset 2** (conectado ao jogo): gerado com o algoritmo MCTS.
     * Deve conter pares (estado atual, melhor jogada).
     * A árvore gerada deve prever a próxima jogada.





## /ai

##### folder com códigos relacionados à IA

### mcts.py

#####
O algoritmo MCTS explora diferentes caminhos de jogadas simuladas. Ele testa jogadas novas, e repete jogadas que deram certo. A heurística ajuda a guiar jogadas em simulações. A melhor jogada é escolhida com base na taxa de sucesso das simulações.

#### Importações
- Importa bibliotecas padrão para tempo, matemática e aleatoriedade.
- Importa arquivos do projeto:
  - `constants`: constantes como `PLAYER1_PIECE` e `PLAYER2_PIECE`.
  - `rules`: lógica do jogo (jogadas válidas, movimentos vencedores, simulação de jogadas, etc).
  - `heuristic`: funções para calcular a pontuação do tabuleiro (heurísticas).

#### Classe Node
Representa um nó da árvore de busca, ou seja, um possível estado do jogo.
- Atributos principais:
  - `board`: o estado do tabuleiro.
  - `parent`: nó pai (estado anterior).
  - `children`: lista de filhos (jogadas a partir do estado atual).
  - `visits`: quantas vezes esse nó foi visitado.
  - `wins`: quantas vitórias vieram desse nó.
  - `current_player`: jogador atual.

- Métodos:
  - `add_children()`: cria filhos simulando todas as jogadas válidas.
  - `select_children()`: retorna até 4 filhos aleatórios.
  - `ucb()`: calcula o *Upper Confidence Bound*, que balanceia as explorações.
  - `score()`: retorna a taxa de vitórias do nó.
  - `__str__()`: imprime estatísticas úteis para debug.

#### Classe MCTS
Implementa o algoritmo **MCTS**, que simula várias partidas e escolhe a melhor jogada.
#### Métodos:
- `__init__(self, root)`
  - Define o nó raiz (estado atual do jogo).

- `start(max_time)`
  - Começa o algoritmo.
  - Gera os filhos do nó raiz.
  - Faz 6 simulações por filho como pré-avaliação.
  - Se algum filho for uma jogada vencedora direta, retorna imediatamente.
  - Caso contrário, chama `search()` para simular por tempo (`max_time` segundos).

- `search(max_time)`
  - Simula até o tempo limite.
  - Para cada iteração:
    - Seleciona um nó folha com `select()`.
    - Se nunca visitado, faz `rollout()` (simulação completa da partida).
    - Senão, expande com novos filhos e simula em cada um deles.
    - Após a simulação, faz `back_propagation()`.

- `select(node)`
  - Navega pela árvore recursivamente até um nó folha, escolhendo sempre o melhor filho com maior `ucb()`.

- `best_child(node)`
  - Retorna o filho com o maior valor de `ucb()`.

- `back_propagation(node, result)`
  - Após uma simulação, atualiza as estatísticas (vitórias e visitas) de todos os nós até a raiz.

- `expand(node)`
  - Expande um nó (gera filhos) e retorna até 4 filhos aleatórios.

- `rollout(node)`
  - Simula uma partida completa a partir de um estado até o fim do jogo.
  - Usa heurísticas para favorecer boas jogadas.
  - Em caso de empate ou falta de informação, joga aleatoriamente.

- `best_move()`
  - Escolhe o melhor movimento com base na maior taxa de vitórias entre os filhos da raiz.
  - Em caso de empate, escolhe aleatoriamente entre os melhores.

#### Função principal `mcts(board)`
- Cria o nó raiz com o tabuleiro atual.
- Executa o algoritmo MCTS por 3 segundos.
- Retorna a coluna escolhida como a melhor jogada.
- Também imprime a coluna escolhida (indexada a partir de 1).



In [ ]:
# Importações necessárias
import time, math, numpy as np, random
from game import constants as c  # Constantes do jogo (como valores para PLAYER1, PLAYER2, etc)
from game import rules as game  # Regras do jogo (funções como available_moves, winning_move, etc)
from ai import heuristic as h   # Heurísticas para pontuar tabuleiros simulados
from math import sqrt, log


# Classe Node representa um estado do jogo (nó na árvore de MCTS)
class Node:
    def __init__(self, board, last_player, parent=None) -> None:
        self.board = board                          # Tabuleiro atual
        self.parent = parent                        # Nó pai (nó anterior à jogada atual)
        self.children = []                          # Lista de filhos (jogadas possíveis a partir deste estado)
        self.visits = 0                             # Número de vezes que este nó foi visitado
        self.wins = 0                               # Número de vitórias associadas a este nó
        self.current_player = 1 if last_player == 2 else 2  # Define o jogador atual (alterna entre 1 e 2)

    def __str__(self) -> str:
        # Representação textual do nó para debug
        string = "Vitórias: " + str(self.wins) + '\n'
        string += "Total: " + str(self.visits) + '\n'
        string += "Pontuação: " + str(self.ucb()) + '\n'
        string += "Probabilidade de vitória: " + str(self.score()) + '\n'
        return string

    def add_children(self) -> None:
        """Adiciona todos os filhos possíveis (jogadas válidas) ao nó atual"""
        if (len(self.children) != 0) or (game.available_moves(self.board) == -1):
            return  # Não adiciona filhos se já existem ou se não há jogadas possíveis

        for col in game.available_moves(self.board):
            # Simula a jogada na coluna col para o jogador atual
            if self.current_player != c.PLAYER2_PIECE:
                copy_board = game.simulate_move(self.board, c.PLAYER2_PIECE, col)
            else:
                copy_board = game.simulate_move(self.board, c.PLAYER1_PIECE, col)

            # Cria novo nó filho com o tabuleiro simulado
            self.children.append((Node(board=copy_board, last_player=self.current_player, parent=self), col))

    def select_children(self):
        """Seleciona aleatoriamente até 4 filhos para simular"""
        if (len(self.children) > 4):
            return random.sample(self.children, 4)
        return self.children

    def ucb(self) -> float:
        """Calcula o Upper Confidence Bound (exploração + exploração) para o nó"""
        if self.visits == 0:
            return float('inf')  # Nós não visitados são priorizados
        exploitation = self.wins / self.visits
        exploration = sqrt(2) * sqrt(2 * log(self.parent.visits / self.visits, math.e)) if self.parent else 0
        return exploitation + exploration

    def score(self) -> float:
        """Retorna a taxa de vitórias (exploração)"""
        if self.visits == 0:
            return 0
        return self.wins / self.visits


# Algoritmo Monte Carlo Tree Search (MCTS)
class MCTS:
    def __init__(self, root: Node) -> None:
        self.root = root

    def start(self, max_time: int):           
        """Inicia o algoritmo: simula 6 vezes por filho antes de começar o MCTS principal"""
        self.root.add_children()

        # Pré-simulações: 6 rollouts por filho
        for child in self.root.children:
            # Se alguma jogada já for uma jogada vencedora imediata, retorna ela
            if game.winning_move(child[0].board, c.PLAYER2_PIECE):
                return child[1]
            for _ in range(6):
                result = self.rollout(child[0])
                self.back_propagation(child[0], result)

        # Inicia MCTS propriamente dito
        return self.search(max_time)

    def search(self, max_time: int) -> int:
        """Busca baseada em tempo (executa MCTS até acabar o tempo)"""
        start_time = time.time()
        while time.time() - start_time < max_time:
            selected_node = self.select(self.root)
            if selected_node.visits == 0:
                result = self.rollout(selected_node)
                self.back_propagation(selected_node, result)
            else:
                selected_children = self.expand(selected_node)
                for child in selected_children:
                    result = self.rollout(child[0])
                    self.back_propagation(child[0], result)
        return self.best_move()

    def select(self, node: Node) -> Node:
        """Seleciona recursivamente o melhor nó folha baseado no UCB"""
        if node.children == []:
            return node
        else:
            node = self.best_child(node)
            return self.select(node)

    def best_child(self, node: Node) -> Node:
        """Retorna o filho com maior UCB"""
        best_child = None
        best_score = float('-inf')
        for (child, _) in node.children:
            ucb = child.ucb()
            if ucb > best_score:
                best_child = child
                best_score = ucb
        return best_child

    def back_propagation(self, node: Node, result: int) -> None:
        """Atualiza estatísticas do nó e seus antecessores"""
        while node:
            node.visits += 1
            if node.current_player == result:  # Se o jogador atual ganhou a simulação
                node.wins += 1
            node = node.parent

    def expand(self, node: Node):
        """Expande o nó atual e retorna até 4 filhos selecionados aleatoriamente"""
        node.add_children()
        return node.select_children()

    def rollout(self, node: Node) -> int:
        """Simula uma partida completa a partir do estado atual até o fim"""
        board = node.board.copy()
        current_player = node.current_player

        while True:
            if game.winning_move(board, 1):
                return 1
            if game.winning_move(board, 2):
                return 2
            if game.is_game_tied(board):
                return 0

            moves = game.available_moves(board)
            if moves == -1:
                return 0

            move_scores = []
            for move in moves:
                sim_board = game.simulate_move(board, current_player, move)
                if game.winning_move(sim_board, current_player):
                    return current_player  # Ganhou diretamente
                # Avalia o tabuleiro usando heurística
                score = h.calculate_board_score(sim_board, current_player, 1 if current_player == 2 else 2)
                move_scores.append(score)

            total = sum(move_scores)
            if total <= 0:
                move = random.choice(moves)
            else:
                weights = [s / total for s in move_scores]
                move = random.choices(moves, weights=weights, k=1)[0]

            # Atualiza tabuleiro e alterna jogador
            board = game.simulate_move(board, current_player, move)
            current_player = 1 if current_player == 2 else 2

    def best_move(self) -> int:
        """Escolhe a melhor jogada com base na maior taxa de vitórias"""
        max_score = float('-inf')
        scores = {}
        columns = []

        for (child, col) in self.root.children:
            score = child.score()
            print(f"Coluna: {col}")
            print(child)
            if score > max_score:
                max_score = score
            scores[col] = score

        for col, score in scores.items():
            if score == max_score:
                columns.append(col)

        return random.choice(columns)  # Em caso de empate, escolhe aleatoriamente


# Função que executa o algoritmo dado um tabuleiro
def mcts(board: np.ndarray) -> int:
    """Retorna a melhor jogada para o tabuleiro dado"""
    root = Node(board=board, last_player=c.PLAYER2_PIECE)
    mcts = MCTS(root)
    column = mcts.start(3)  # Executa por 3 segundos
    print(column + 1)  # Para debug (colunas indexadas a partir de 1)
    return column


### decision_tree.py

#####
Esse código implementa a árvore de decisão personalizada usada para: Classificação do dataset Iris, e a decisão de jogadas do jogo com base na dataset gerada por generate_dataset.

#### 1. Importações
`pandas`, `numpy`, `random`: para manipulação de dados e operações matemáticas.
`joblib`: para salvar e carregar o modelo treinado.
`game.rules` e `game.constants`: módulos personalizados com as regras e constantes do jogo Connect4.

#### 2. Classe `DTNode`
Representa um nó da árvore de decisão. Cada nó pode conter:
- O índice e nome do atributo usado para divisão.
- Dicionário de filhos (caso não seja um nó folha).
- Valor de ganho de informação.
- Lista de valores do atributo usados para dividir.
- Valor da classe, se for um nó folha.

#### 3. Classe `DecisionTreeClassifier`
Implementa uma árvore de decisão com os seguintes recursos:

#### Inicialização
Parâmetros:
- `max_depth`: profundidade máxima da árvore.
- `min_samples_split`: número mínimo de amostras para dividir um nó.
- `criterium`: critério para medir impureza (`entropy` ou `gini`).

#### Métodos principais
- `fit(X_train, y_train)`: junta os dados com os rótulos e inicia a construção da árvore.
- `_build_tree(dataset, curr_depth)`: constrói a árvore de forma recursiva, usando o melhor split.
- `_calculate_leaf_value(y_train)`: define o valor de um nó folha como o valor mais comum.
- `_is_pure(target_column)`: verifica se todas as classes são iguais (puro).
- `_get_best_split(dataset)`: encontra o melhor atributo para divisão com maior ganho de informação.
- `_discrete_split(dataset, feature_index)`: divide o dataset de acordo com os valores únicos de uma feature.
- `_discrete_info_gain(y_train, splits)`: calcula o ganho de informação de uma divisão.
- `_get_impurity(y_train)`: chama `entropy` ou `gini_index` conforme critério.
- `_entropy(y_train)`: mede a entropia de uma coluna.
- `_gini_index(y_train)`: mede o índice de Gini.
- `predict(X_test)`: faz previsões para cada linha do DataFrame.
- `make_prediction(row, node)`: percorre a árvore para prever o valor da classe de uma única linha.

#### 4. Classe `DecisionTree`
Encapsula o uso da árvore para diferentes domínios (Iris e Connect4).

#### Inicialização
- `mode`: define o modo de operação (`iris` ou `connect4`).
- `initialize_model()`: carrega ou treina o modelo dependendo do modo.

#### Métodos para cada dataset
- `_initialize_iris_model()`: tenta carregar o modelo salvo ou treina com o arquivo `iris.csv`.
- `_initialize_connect4_model()`: idem, mas com `connect4.csv`.

#### Predição
- `predict_iris(...)`: recebe atributos da flor e retorna a classe prevista.
- `play(board)`: escolhe a melhor jogada para o jogador 1 no Connect4:
  - Gera todas as jogadas possíveis.
  - Simula os estados dos tabuleiros após cada jogada.
  - Transforma os estados em DataFrame.
  - Faz a previsão de vitória, empate ou derrota.
  - Prioriza jogadas vencedoras, depois empates, e evita derrotas.





In [ ]:
# Importações necessárias
from typing import Union
import numpy as np
import pandas as pd
import random
from pandas import DataFrame, Series, read_csv
from game import rules as game        # Importa regras do jogo Connect4
from game import constants as c       
from joblib import load, dump         # Para salvar e carregar modelos com persistência


# Classe que representa um nó da árvore de decisão
class DTNode:
    def __init__(self, feature_index=None, feature_name=None, children=None, info_gain=None, split_values=None, leaf_value=None) -> None:
        self.feature_index = feature_index      # Índice do atributo usado para o split
        self.feature_name = feature_name        # Nome do atributo
        self.children = children                # Dicionário de nós filhos
        self.info_gain = info_gain              # Ganho de informação do split
        self.split_values = split_values        # Valores do atributo para cada ramo
        self.leaf_value = leaf_value            # Valor da classe se o nó for uma folha

# Classe que constrói e faz previsões com árvore de decisão
class DecisionTreeClassifier:
    def __init__(self, max_depth: int = None, min_samples_split: int = None, criterium: str = 'entropy') -> None:
        self.root = None                        # Nó raiz da árvore
        self.max_depth = max_depth              # Profundidade máxima permitida
        self.min_samples_split = min_samples_split  # Número mínimo de amostras para dividir
        self.criterium = criterium              # Critério de impureza (gini ou entropia)

    # Treinamento da árvore
    def fit(self, X_train: DataFrame, y_train: Series) -> None:
        dataset = pd.concat((X_train, y_train), axis=1)  # Une dados e rótulos
        self.root = self._build_tree(dataset)            # Constrói a árvore recursivamente

    # Constrói a árvore de forma recursiva
    def _build_tree(self, dataset: DataFrame, curr_depth: int = 0) -> DTNode:
        X_train = dataset.iloc[:, :-1]
        y_train = dataset.iloc[:, -1]
        num_samples, num_features = X_train.shape

        # Condições de parada da recursão (puro, profundidade, amostras mínimas)
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth and not self._is_pure(y_train):
            best_split = self._get_best_split(dataset)
            if best_split["info_gain"] > 0:
                children = {
                    feature_value: self._build_tree(subset, curr_depth + 1)
                    for feature_value, subset in best_split["splits"].items()
                }
                return DTNode(best_split["feature_index"], best_split["feature_name"], children, best_split["info_gain"], best_split["splits"].keys())

        # Caso base: retorna um nó folha com o valor mais comum
        leaf_value = self._calculate_leaf_value(y_train)
        return DTNode(leaf_value=leaf_value)

    # Retorna o valor mais comum (modo) para usar como folha
    def _calculate_leaf_value(self, y_train: Series) -> any:
        y_train = list(y_train)
        return max(y_train, key=y_train.count)

    # Verifica se todos os valores da coluna são iguais
    def _is_pure(self, target_column: Series) -> bool:
        return len(set(target_column)) == 1

    # Encontra o melhor atributo e divisão dos dados
    def _get_best_split(self, dataset: DataFrame) -> dict:
        best_split = {}
        max_info_gain = -float("inf")
        num_features = dataset.shape[1] - 1

        for feature_index in range(num_features):
            feature_name = dataset.columns[feature_index]
            feature_values = dataset.iloc[:, feature_index]
            splits = self._discrete_split(dataset, feature_index)

            # Calcula o ganho de informação da divisão
            info_gain = self._discrete_info_gain(dataset.iloc[:, -1], splits)

            # Atualiza se for o melhor ganho encontrado
            if info_gain > max_info_gain:
                best_split = self._update_best_split(feature_index, feature_name, info_gain, splits)
                max_info_gain = info_gain

        return best_split

    # Atualiza o dicionário com as melhores informações de split
    def _update_best_split(self, feature_index, feature_name, info_gain, splits) -> dict:
        return {
            "feature_index": feature_index,
            "feature_name": feature_name,
            "info_gain": info_gain,
            "splits": splits,
        }

    # Realiza split discreto dos dados por valor da feature
    def _discrete_split(self, dataset: DataFrame, feature_index: int) -> dict:
        splits = {}
        for feature_value in dataset.iloc[:, feature_index].unique():
            splits[feature_value] = dataset[dataset.iloc[:, feature_index] == feature_value]
        return splits

    # Calcula o ganho de informação com base nos splits
    def _discrete_info_gain(self, y_train: Series, splits: dict) -> float:
        weight_average = sum((len(subset) / len(y_train)) * self._get_impurity(subset.iloc[:, -1]) for subset in splits.values())
        return self._get_impurity(y_train) - weight_average

    # Retorna a impureza segundo o critério escolhido
    def _get_impurity(self, y_train: Series) -> float:
        return self._entropy(y_train) if self.criterium == "entropy" else self._gini_index(y_train)

    # Cálculo da entropia
    def _entropy(self, y_train: Series) -> float:
        class_labels = list(set(y_train))
        probabilities = [y_train.tolist().count(label) / len(y_train) for label in class_labels]
        return -sum(p * np.log2(p) for p in probabilities if p > 0)

    # Cálculo do índice de Gini
    def _gini_index(self, y_train: Series) -> float:
        class_labels = list(set(y_train))
        probabilities = [y_train.tolist().count(label) / len(y_train) for label in class_labels]
        return 1 - sum(p ** 2 for p in probabilities)

    # Previsões para um DataFrame de dados
    def predict(self, X_test: DataFrame) -> list:
        return [self.make_prediction(row, self.root) for _, row in X_test.iterrows()]

    # Faz a previsão para uma única amostra
    def make_prediction(self, row: tuple, node: DTNode) -> Union[any, None]:
        while node and node.leaf_value is None:
            value = row[node.feature_index]
            if value in node.children:
                node = node.children[value]
            else:
                return None  # Valor não esperado na árvore
        return node.leaf_value
    
    def predict_connect4_move(self, board: np.ndarray) -> int:
        possible_moves = game.available_moves(board)
        if not possible_moves:
            return -1  # Jogo empatado

        # Converte o tabuleiro para um formato compatível com o modelo
        flattened = board.flatten()
        df = pd.DataFrame([flattened])
        prediction = self.clf.predict(df)[0]

        # Lógica simplificada: escolhe a primeira jogada válida com melhor resultado
        for move in possible_moves:
            sim_board = game.simulate_move(board, c.PLAYER2_PIECE, move)
            sim_flattened = sim_board.flatten()
            sim_df = pd.DataFrame([sim_flattened])
            if self.clf.predict(sim_df)[0] == prediction:
                return move
        return possible_moves[0]

# Classe que encapsula o uso da árvore para diferentes domínios
class DecisionTree:
    def __init__(self, mode='iris'):
        self.mode = mode  # Modo: 'iris' ou 'connect4'
        self.clf = None   # Classificador
        self.initialize_model()

    # Inicializa o modelo conforme o modo
    def initialize_model(self):
        if self.mode == 'iris':
            self._initialize_iris_model()
        elif self.mode == 'connect4':
            self._initialize_connect4_model()

    # Inicializa modelo para o dataset Iris
    def _initialize_iris_model(self):
        try:
            self.clf = load("datasets/iris_model.joblib")
        except FileNotFoundError:
            df = read_csv("datasets/iris.csv")
            X = df.iloc[:, :-1]
            y = df.iloc[:, -1]
            self.clf = DecisionTreeClassifier(3, 2, "entropy")
            self.clf.fit(X, y)
            dump(self.clf, "datasets/iris_model.joblib")

    # Inicializa modelo para o jogo Connect4
    def _initialize_connect4_model(self):
        try:
            self.clf = load("datasets/connect4_model.joblib")
        except FileNotFoundError:
            df = read_csv("datasets/connect4_dataset.csv")
            X = df.iloc[:, :-1]
            y = df.iloc[:, -1]
            self.clf = DecisionTreeClassifier(5, 2, "entropy")
            self.clf.fit(X, y)
            dump(self.clf, "datasets/connect4_model.joblib")

    # Faz a previsão para os dados da íris
    def predict_iris(self, sepal_length, sepal_width, petal_length, petal_width):
        X_test = pd.DataFrame([[sepal_length, sepal_width, petal_length, petal_width]])
        return self.clf.predict(X_test)[0]

    # Decide a melhor jogada para o jogo Connect4
    def play(self, board):
        possible_plays = game.get_possible_plays(board)
        possible_states = []
        for play in possible_plays:
            new_board = game.make_play(play, c.PLAYER1, board)
            possible_states.append((play, new_board))

        board_dicts = [game.to_dict(board) for _, board in possible_states]
        df = pd.DataFrame(board_dicts)
        predictions = self.clf.predict(df)

        # Cria um dicionário de colunas por previsão
        prediction_dict = {}
        for i, prediction in enumerate(predictions):
            if prediction not in prediction_dict:
                prediction_dict[prediction] = []
            prediction_dict[prediction].append(possible_states[i][0])

        # Prioriza vitória, depois empate, depois evitar derrota
        if c.WIN in prediction_dict:
            return random.choice(prediction_dict[c.WIN])
        elif c.DRAW in prediction_dict:
            return random.choice(prediction_dict[c.DRAW])
        elif c.LOSS in prediction_dict:
            return random.choice(prediction_dict[c.LOSS])
        else:
            return random.choice(possible_plays)



### heuristic.py

#####

Este módulo implementa a função de avaliação heurística do jogo. Ele percorre o tabuleiro em todas as direções e analisa grupos de 4 posições, atribuindo pontuações que ajudam a IA a tomar decisões estratégicas. A função calculate_board_score é usada para comparar possíveis movimentos e escolher o melhor caminho ofensivo ou defensivo.


#### 1. Importações:
- from game import constants as c: importa as constantes do jogo, como número de linhas e colunas (c.ROWS, c.COLUMNS).
- import numpy as np: importa o NumPy para manipulação de arrays, onde o tabuleiro é representado como uma matriz 2D.

#### 2. Função `calculate_board_score(board, piece, opponent_piece)`
Esta função avalia o tabuleiro atual e retorna uma pontuação numérica que indica o quão favorável ele está para o jogador especificado.

Como funciona:
- Horizontal: percorre cada linha e avalia grupos de 4 colunas consecutivas.
- Vertical: percorre cada coluna e avalia grupos de 4 linhas consecutivas.
- Diagonal crescente (↘): avalia diagonais que vão da parte superior esquerda para a inferior direita.
- Diagonal decrescente (↗): avalia diagonais que vão da parte inferior esquerda para a superior direita.

Cada segmento de 4 posições é passado para a função weights(), que calcula o valor estratégico do grupo.

#### 3. Função `weights(segment, piece, opponent_piece)`
Esta função recebe uma lista de 4 posições e atribui uma pontuação com base no conteúdo:

- Se houver peças dos dois jogadores no segmento, retorna 0 (sem vantagem).
- Se houver peças apenas do jogador:
  - 1 peça: +1
  - 2 peças: +10
  - 3 peças: +50
  - 4 peças: +1000 (situação de vitória)
- Se houver peças apenas do oponente:
  - 1 peça: -1
  - 2 peças: -10
  - 3 peças: -50
  - 4 peças: -2000 (situação crítica a evitar)

Com isso, a IA consegue avaliar tanto as oportunidades de vitória quanto as ameaças do oponente.




In [ ]:
from game import constants as c
import numpy as np

def calculate_board_score(board: np.ndarray, piece: int, opponent_piece: int) -> int:
    score = 0

    # Check horizontal
    for col in range(c.COLUMNS - 3):
        for r in range(c.ROWS):
            segment = [board[r][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Check vertical
    for col in range(c.COLUMNS):
        for r in range(c.ROWS - 3):
            segment = [board[r + i][col] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Check ascending diagonal
    for col in range(c.COLUMNS - 3):
        for r in range(c.ROWS - 3):
            segment = [board[r + i][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Check descending diagonal
    for col in range(c.COLUMNS - 3):
        for r in range(3, c.ROWS):
            segment = [board[r - i][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    return score


def weights(segment: list, piece: int, opponent_piece: int) -> int:
    if piece in segment and opponent_piece in segment: return 0
    if segment.count(piece) == 1: return 1
    if segment.count(piece) == 2: return 10
    if segment.count(piece) == 3: return 50
    if segment.count(piece) == 4: return 1000
    if segment.count(opponent_piece) == 1: return -1
    if segment.count(opponent_piece) == 2: return -10
    if segment.count(opponent_piece) == 3: return -50
    if segment.count(opponent_piece) == 4: return -2000
    return 0

## /game

##### folder com códigos relacionados ao jogo em si, e suas regras e verificações

### board.py

#####
Este arquivo define a classe `Board`, que representa o tabuleiro do jogo Connect 4. Ele utiliza a biblioteca NumPy para criar e manipular a matriz do tabuleiro, basicamente é o que cria e mostra o tabuleiro.

#### Importações:
- `numpy as np`: Biblioteca usada para operações com arrays.
- `constants as c`: Importa constantes como número de linhas e colunas.
- `dataclass` e `field`: Usados para definir classes com menos código boilerplate.

#### Classe `Board`:
- Esta classe é um dataclass que representa o estado do tabuleiro do jogo.

#### Atributos:
- `rows`: Número de linhas do tabuleiro (padrão definido em `constants.py`).
- `columns`: Número de colunas do tabuleiro (padrão definido em `constants.py`).
- `board`: Matriz 2D do NumPy que representa o tabuleiro. Inicializada como uma matriz 6x7 preenchida com zeros (representando um tabuleiro vazio).

#### Métodos:
- `get_board()`: Retorna o estado atual do tabuleiro (a matriz NumPy).
- `print_board()`: Imprime o tabuleiro no console, invertendo a ordem das linhas para que a linha inferior do jogo apareça embaixo (como numa visualização real).



In [ ]:
import numpy as np
from game import constants as c
from dataclasses import dataclass, field

@dataclass
class Board:
    rows: int = c.ROWS
    columns: int = c.COLUMNS
    board: np.ndarray = field(default_factory=lambda: np.zeros((6,7)))
            
    def get_board(self) -> np.ndarray:
        return self.board

    def print_board(self) -> None:
        print(np.flip(self.board, 0), "\n")
        
 

### rules.py

##### 
Este módulo implementa toda a lógica das jogadas do jogo, tanto para humanos quanto para IA. Ele cuida da detecção de vitória, empate, jogadas válidas e atualização da interface visual com Pygame. Ele também decide qual IA utilizar de acordo com o modo de jogo, e as chama, integrando inteligência artificial para competir contra o jogador.


#### 1. Importações:
   - game.constants as c: importa constantes do jogo (dimensões, cores, número de colunas, peças, etc.).
   - numpy: manipulação de arrays, essencial para representar o tabuleiro.
   - math: funções matemáticas, usada por exemplo para arredondar a posição do clique.
   - pygame: biblioteca para criar a interface gráfica.
   - game.board import Board: importa a classe do tabuleiro.
   - ai import a_star, mcts, decision_tree: algoritmos de IA usados para jogadas automáticas.

#### 2. Função `human_move(...)`
   - Determina a coluna onde o jogador humano clicou.
   - Valida se a jogada é possível (coluna não cheia ou inválida).
   - Atualiza a interface e realiza o movimento se for válido.

#### 3. Função `get_human_column(...)`
   - Traduz a coordenada x do clique do mouse para uma coluna válida do tabuleiro.

#### 4. Função `available_moves(...)`
   - Retorna uma lista com todas as colunas onde ainda é possível jogar.
   - Retorna -1 se nenhuma coluna estiver disponível (jogo travado ou empate).

#### 5. Função `ai_move(...)`
   - Comanda a jogada da IA.
   - Escolhe a coluna usando um dos algoritmos de IA.
   - Realiza o movimento e retorna se o jogo terminou.

#### 6. Função `get_ai_column(...)`
   - Define qual IA será usada de acordo com o modo de jogo:
     - Modo 2: MCTS.
     - Modo 3: alterna entre MCTS (jogador X) e A* (jogador O).

#### 7. Função `simulate_move(...)`
   - Faz uma cópia do tabuleiro e simula um movimento, sem alterar o estado real.
   - Usado geralmente para IA prever futuros estados.

#### 8. Função `make_move(...)`
   - Realiza efetivamente o movimento no tabuleiro.
   - Desenha a nova peça na interface gráfica.
   - Verifica se houve vitória ou empate após o movimento.

#### 9. Função `get_next_open_row(...)`
   - Retorna a primeira linha disponível em uma coluna, de baixo para cima.

#### 10. Função `drop_piece(...)`
   - Atualiza a matriz do tabuleiro com a peça na posição correta.

#### 11. Função `is_game_tied(...)`
   - Verifica se o tabuleiro está completamente cheio e não houve vitória, indicando empate.

#### 12. Função `is_valid(...)`
   - Valida se a coluna escolhida está dentro do intervalo e se ainda tem espaço para jogar.

#### 13. Função `winning_move(...)`
   - Verifica se houve vitória com a jogada atual.
   - Implementa verificações nas quatro direções:
      - Horizontal.
      - Vertical.
      - Diagonal crescente (↘).
      - Diagonal decrescente (↗).
    - Retorna True se qualquer uma das verificações indicar quatro peças consecutivas do mesmo jogador.




In [ ]:
# Importação de bibliotecas e módulos necessários
import game.constants as c  # Constantes do jogo, como dimensões, cores e peças
import numpy as np  # Biblioteca para manipulação de arrays (tabuleiro)
import math  # Biblioteca para funções matemáticas
import pygame  # Biblioteca para interface gráfica
from game.board import Board  # Classe que representa o tabuleiro
from ai import decision_tree as tree, mcts as m  # Importação dos algoritmos de IA

# Função responsável pela jogada do jogador humano
def human_move(bd: Board, interface: any, board: np.ndarray, turn: int, event: any) -> bool:
    """Define a coluna onde o jogador humano jogou"""
    col = get_human_column(interface, event)  # Obtém a coluna a partir do clique do mouse
    if not is_valid(board, col): 
        return False  # Movimento inválido se a coluna estiver cheia ou fora do tabuleiro

    # Atualiza a interface apagando a linha superior
    pygame.draw.rect(interface.screen, c.BACKGROUND_COLOR, (0,0, interface.width, interface.pixels-14))   
    make_move(bd, interface, board, turn, col)  # Realiza o movimento
    return True  # Movimento válido

# Função que obtém a coluna clicada pelo jogador com base na posição do mouse
def get_human_column(interface: any, event: any):
    posx = event.pos[0]  # Posição X do mouse
    col = int(math.floor(posx / interface.pixels)) - 2  # Traduz coordenada para coluna
    return col

# Retorna uma lista com todas as colunas que ainda aceitam peças
def available_moves(board: np.ndarray) -> list | int:
    avaiable_moves = []
    for i in range(c.COLUMNS):
        if board[5][i] == 0:  # Verifica a linha do topo
            avaiable_moves.append(i)
    return avaiable_moves if len(avaiable_moves) > 0 else -1

# Função principal que comanda a jogada da IA
def ai_move(bd: Board, interface: any, game_mode: int, board: np.ndarray, turn: int) -> int:
    """Define a coluna onde a IA vai jogar"""
    ai_column = get_ai_column(board, game_mode)  # Escolhe a jogada com base no algoritmo
    game_over = make_move(bd, interface, board, turn, ai_column)  # Realiza a jogada
    return game_over  # Retorna se o jogo terminou

# Função que decide qual algoritmo de IA será usado de acordo com o modo de jogo
def get_ai_column(board: np.ndarray, game_mode: int) -> int:
    if game_mode == 2:  
        return m.mcts(board)  # Modo 2 usa Monte Carlo Tree Search
    elif game_mode == 3:  
        # Alternância entre MCTS e A* baseado no número de peças
        x_count = np.count_nonzero(board == 1)
        o_count = np.count_nonzero(board == 2)
        if x_count == o_count:  # Vez do X (MCTS)
            return m.mcts(board)
        else:  # Vez do O (A*)
            return tree.DecisionTree(mode='connect4').play(board)

# Simula um movimento sem alterar o tabuleiro real
def simulate_move(board: np.ndarray, piece: int, col: int) -> np.ndarray:
    board_copy = board.copy()
    row = get_next_open_row(board_copy, col)
    drop_piece(board_copy, row, col, piece)
    return board_copy

# Executa o movimento no tabuleiro e desenha na tela
def make_move(bd: Board, interface: any, board: np.ndarray, turn: int, move: int):
    row = get_next_open_row(board, move)  # Encontra a linha disponível
    drop_piece(board, row, move, turn)  # Atualiza o tabuleiro
    interface.draw_new_piece(row+1, move+2, turn)  # Desenha na interface
    pygame.display.update()  # Atualiza a tela
    bd.print_board()  # Imprime o tabuleiro no terminal (debug)

    # Verifica se o jogo terminou com vitória ou empate
    return winning_move(board, turn) or is_game_tied(board)

# Retorna a primeira linha disponível em uma coluna
def get_next_open_row(board: np.ndarray, col: int) -> int:
    for row in range(c.ROWS):
        if board[row][col] == 0:
            return row
    return -1  # Nenhuma linha disponível

# Insere a peça no tabuleiro
def drop_piece(board: np.ndarray, row: int, col: int, piece: int) -> None:
    board[row][col] = piece

# Verifica se o jogo empatou (tabuleiro cheio sem vencedores)
def is_game_tied(board: np.ndarray) -> bool:
    if winning_move(board, c.PLAYER1_PIECE) or winning_move(board, c.PLAYER2_PIECE): 
        return False
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0: 
                return False  # Ainda há jogadas possíveis
    return True  # Tabuleiro cheio e sem vencedor

# Verifica se a coluna selecionada é válida
def is_valid(board: np.ndarray, col: int) -> bool:
    if not 0 <= col < c.COLUMNS: 
        return False  # Fora do intervalo de colunas
    row = get_next_open_row(board, col)
    return 0 <= row <= 5  # Há uma linha disponível

# Verifica se o movimento atual resulta em vitória
def winning_move(board: np.ndarray, piece: int) -> bool:
    """Verifica todas as direções possíveis de vitória"""
    
    # Horizontal
    def check_horizontal(board: np.ndarray, piece: int) -> bool:
        for col in range(c.COLUMNS - 3):
            for row in range(c.ROWS):
                if all(board[row][col+i] == piece for i in range(4)):
                    return True

    # Vertical
    def check_vertical(board: np.ndarray, piece: int) -> bool:
        for col in range(c.COLUMNS):
            for row in range(c.ROWS - 3):
                if all(board[row+i][col] == piece for i in range(4)):
                    return True

    # Diagonal crescente
    def check_ascending_diagonal(board: np.ndarray, piece: int) -> bool:
        for col in range(c.COLUMNS - 3):
            for row in range(c.ROWS - 3):
                if all(board[row+i][col+i] == piece for i in range(4)):
                    return True

    # Diagonal decrescente
    def check_descending_diagonal(board: np.ndarray, piece: int) -> bool:
        for col in range(c.COLUMNS - 3):
            for row in range(3, c.ROWS):
                if all(board[row-i][col+i] == piece for i in range(4)):
                    return True

    # Retorna verdadeiro se qualquer direção gerar vitória
    return (
        check_vertical(board, piece) or 
        check_horizontal(board, piece) or 
        check_ascending_diagonal(board, piece) or 
        check_descending_diagonal(board, piece)
    )



### constants.py

#####

CONSTANTES DO JOGO

- Este arquivo define características visuais e estruturais do jogo.
- Facilita alterações no tema visual e tamanho da tela.
- Mantém o código principal mais organizado e modular.


In [ ]:
# Colors
BOARD_COLOR = (169, 169, 169)
BACKGROUND_COLOR = (30, 25, 112)
SHADOW_COLOR = (170, 170, 170) 
PLAYER1_COLOR = (195,53,43)
PLAYER2_COLOR = (255,205,49) 
TEXT_COLOR = (255,205,49) 
BUTTON_COLOR = (195,53,43)
BUTTON_HOVER_COLOR = (155,13,0)

PIECES_COLORS = [BACKGROUND_COLOR, PLAYER1_COLOR, PLAYER2_COLOR, PLAYER1_COLOR, PLAYER2_COLOR]
PLAYER1_PIECE = 1
PLAYER2_PIECE = 2


# Constants for the data matrix
ROWS = 6
COLUMNS = 7

# Constants for the board image
SQUARESIZE = 100    # size of each square that will divide the screen 
RADIUS = 43     # radius of each player piece

# Constants for the interface (scren)
WIDTH = (4+COLUMNS) * SQUARESIZE    # width of the screen = board + 2 empty columns on each side of the screen
HEIGHT = (ROWS+2) * SQUARESIZE    # height of the screen = board + 1 empty row under and above the board
   
        

## /interface

##### folder com a interface

### interface.py

##### 

Apesar da interface grágica não ser exigida, decidimos por fazer e deixar nosso trabalho mais apresentável. O arquivo é responsável por exibir a tela do jogo, o menu inicial, interações do usuário (movimentos com o mouse e cliques), além de exibir mensagens de vitória ou empate.

#### 1. Classe Interface

Utiliza o decorator `@dataclass` para simplificar a inicialização com valores padrão, como número de linhas, colunas, tamanho dos quadrados, etc., todos definidos em um módulo de constantes `constants`.

#### 2. Atributos Principais

- `screen`: janela principal do pygame.
- `size`: tupla com largura e altura da tela.
- `rad`: raio das peças.

#### 3. start_game(self, bd)

- Inicializa o pygame e desenha o menu inicial de opções.
- Chama `choose_option()` para selecionar o modo de jogo.
- Chama `draw_board()` para desenhar o tabuleiro vazio.
- Inicia o jogo com `play_game()`.

#### 4. play_game(self, bd, game_mode)

Executa o loop principal do jogo:
- Detecta eventos do mouse e QUIT.
- Permite jogadas humanas se for um modo com jogador humano.
- Usa IA nos modos onde o turno da máquina está ativo.
- Verifica vitórias e empates com funções do módulo `rules`.
- Exibe o vencedor ou mensagem de empate e espera 5 segundos.

#### 5. draw_options_board(self)

- Desenha o menu inicial com título “Connect 4”.
- Mostra três botões:
  - Player x Player
  - Player x MCTS
  - A* x MCTS

#### 6. choose_option(self)

- Loop de seleção de modo de jogo.
- Detecta o hover do mouse em cada botão.
- Retorna 1, 2 ou 3 dependendo do botão clicado.

#### 7. draw_board(self)

- Desenha o tabuleiro de jogo com sombra e estilo.
- Adiciona círculos vazios para cada posição do tabuleiro.

#### 8. draw_new_piece(self, row, col, piece)

- Desenha uma nova peça (círculo colorido) na posição dada.

#### 9. draw_button(self, x, y, width, height, text, hovered=False)

- Desenha um botão com texto.
- Muda a cor do botão se o mouse estiver sobre ele.

#### 10. show_winner(self, myfont, turn)

- Mostra na tela uma mensagem de vitória com a cor do jogador vencedor.

#### 11. show_draw(self, myfont)

- Mostra na tela a mensagem "Game tied!" em caso de empate.

#### 12. quit()

- Encerra o pygame e o programa completamente.

##### Observações Finais

Este módulo serve como a ponte entre o jogador (usuário) e a lógica do jogo, provendo uma interface visual atrativa e interativa. Ele depende de outros arquivos para aplicar as regras do jogo e IA, e atualizar o estado do jogo.



In [ ]:
import pygame 
import itertools
import sys
from game import constants as c
from game.board import Board
import game.rules as game
from dataclasses import dataclass


@dataclass
class Interface:
    # Parâmetros padrão definidos a partir de constantes
    rows: int = c.ROWS
    columns: int = c.COLUMNS
    pixels: int = c.SQUARESIZE
    width: int = c.WIDTH
    height: int = c.HEIGHT
    rad: float = c.RADIUS
    size: tuple = (width, height)
    screen: any = pygame.display.set_mode(size)  # Janela do pygame

    pygame.display.set_caption("Connect4")  # Título da janela


    def start_game(self, bd: Board) -> None:
        """Inicializa o jogo, mostra as opções e começa a execução"""
        pygame.init()
        self.draw_options_board()  # Desenha o menu inicial
        game_mode = self.choose_option()  # Usuário escolhe modo de jogo
        bd.print_board()	
        self.draw_board()    # Desenha o tabuleiro vazio
        pygame.display.update()
        self.play_game(bd, game_mode)  # Começa a partida


    def play_game(self, bd: Board, game_mode: int) -> None:
        """Executa o loop principal do jogo"""
        board = bd.get_board()	
        game_over = False
        myfont = pygame.font.SysFont("Monospace", 50, bold=True)
        turns = itertools.cycle([1, 2])  # Alternância de turnos
        turn = next(turns)
        
        while not game_over:
            for event in pygame.event.get():
                if event.type == pygame.QUIT: 
                    self.quit()

                # Movimento humano (somente nos modos 1 e 2)
                if game_mode != 3:
                    if event.type == pygame.MOUSEMOTION:
                        pygame.draw.rect(self.screen, c.BACKGROUND_COLOR, (0, 0, self.width, self.pixels - 14))
                        posx = event.pos[0]
                        pygame.draw.circle(self.screen, c.PIECES_COLORS[turn], (posx, int(self.pixels / 2) - 7), self.rad)
                        pygame.display.update()

                    if event.type == pygame.MOUSEBUTTONDOWN and (turn == 1 or (turn == 2 and game_mode == 1)):
                        if not game.human_move(bd, self, board, turn, event): 
                            continue
                        if game.winning_move(board, turn): 
                            game_over = True
                            break
                        turn = next(turns)

            # Movimento de IA (modo 2 ou 3)
            if (game_mode == 2 and turn == 2) or game_mode == 3:
                pygame.time.wait(500)  # Espera para visualizar jogada
                game_over = game.ai_move(bd, self, game_mode, board, turn)
                if game_over: 
                    break
                turn = next(turns)

            if game.is_game_tied(board):
                self.show_draw(myfont)
                break

        if game.winning_move(board, turn):
            self.show_winner(myfont, turn)

        pygame.time.wait(5000)  # Espera antes de fechar


    def draw_options_board(self):
        """Desenha a tela inicial com os modos de jogo"""
        self.screen.fill(c.BACKGROUND_COLOR)
        font = pygame.font.Font("interface/fonts/FreckleFace-Regular.ttf", 150)        
        text_surface = font.render("Connect 4", True, c.TEXT_COLOR)
        text_rect = text_surface.get_rect(center=(560, 230))
        self.screen.blit(text_surface, text_rect)
        self.draw_button(self.height / 2 - 150, 350, 400, 70, "Player x Player")
        self.draw_button(self.height / 2, 450, 400, 70, "Player x MCTS") 
        self.draw_button(self.height / 2 - 90, 550, 400, 70, "Decision Tree x MCTS") 


    def choose_option(self) -> int:
        """Permite ao jogador escolher o modo de jogo clicando nos botões"""
        while True:
            game_mode = 0
            mouse_x, mouse_y = pygame.mouse.get_pos()

            self.screen.fill(c.BACKGROUND_COLOR)
            font = pygame.font.Font("interface/fonts/FreckleFace-Regular.ttf", 150)        
            text_surface = font.render("Connect 4", True, c.TEXT_COLOR)
            text_rect = text_surface.get_rect(center=(560, 230))
            self.screen.blit(text_surface, text_rect)

            # Detecta hover sobre os botões
            hover_player = (self.height / 2 - 150 <= mouse_x <= self.height / 2 - 150 + 400) and (350 <= mouse_y <= 350 + 70)
            hover_ai = (self.height / 2 <= mouse_x <= self.height / 2 + 400) and (450 <= mouse_y <= 450 + 70)
            hover_ai_vs_ai = (self.height / 2 - 90 <= mouse_x <= self.height / 2 - 90 + 400) and (550 <= mouse_y <= 550 + 70)

            # Redesenha botões com efeito de hover
            self.draw_button(self.height / 2 - 150, 350, 400, 70, "Player x Player", hover_player)
            self.draw_button(self.height / 2, 450, 400, 70, "Player x MCTS", hover_ai)
            self.draw_button(self.height / 2 - 90, 550, 400, 70, "Decision Tree x MCTS", hover_ai_vs_ai)

            for event in pygame.event.get():
                if event.type == pygame.QUIT: 
                    self.quit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    if hover_player:
                        print("Player vs Player selecionado")
                        game_mode = 1
                    elif hover_ai:
                        print("Player vs AI selecionado")
                        game_mode = 2
                    elif hover_ai_vs_ai:
                        print("AI vs AI selecionado")
                        game_mode = 3
            
            pygame.display.flip()

            if game_mode in [1, 2, 3]:
                return game_mode


    def draw_board(self) -> None:
        """Desenha o tabuleiro principal do jogo"""
        self.screen.fill(c.BACKGROUND_COLOR)    

        # Desenha tabuleiro com sombra
        shadow_coordinates = (2*self.pixels - 10, self.pixels - 10, self.columns*self.pixels + 24, self.rows*self.pixels + 24)
        board_coordinates = (2*self.pixels - 10, self.pixels - 10, self.columns*self.pixels + 20, self.rows*self.pixels + 20)
        pygame.draw.rect(self.screen, c.SHADOW_COLOR, shadow_coordinates, 0, 30)
        pygame.draw.rect(self.screen, c.BOARD_COLOR, board_coordinates, 0, 30)

        # Desenha círculos (vazios) no tabuleiro
        for col in range(self.columns):
            for row in range(self.rows):
                center = (int((col + 5 / 2) * self.pixels), int((row + 3 / 2) * self.pixels))
                pygame.draw.circle(self.screen, c.BACKGROUND_COLOR, center, self.rad)
        pygame.display.update()


    def draw_new_piece(self, row: int, col: int, piece: int) -> None:
        """Desenha uma nova peça no tabuleiro"""
        center = (int(col * self.pixels + self.pixels / 2), self.height - int(row * self.pixels + self.pixels / 2))
        pygame.draw.circle(self.screen, c.PIECES_COLORS[piece], center, self.rad)


    def draw_button(self, x: int, y: int, width: int, height: int, text: str, hovered: bool = False) -> None:
        """Desenha um botão interativo com texto"""
        color = c.BUTTON_HOVER_COLOR if hovered else c.BUTTON_COLOR
        pygame.draw.rect(self.screen, color, (x, y, width, height), 0, 30)
        font = pygame.font.Font("interface/fonts/FreckleFace-Regular.ttf", 36)
        text_surface = font.render(text, True, c.TEXT_COLOR)
        text_rect = text_surface.get_rect(center=(x + width / 2, y + height / 2))
        self.screen.blit(text_surface, text_rect)


    def show_winner(self, myfont: any, turn: int) -> None:
        """Exibe mensagem de vitória"""
        label = myfont.render("Player " + str(turn) + " wins!", True, c.PIECES_COLORS[turn])
        self.screen.blit(label, (350, 15))
        pygame.display.update()


    def show_draw(self, myfont: any) -> None:
        """Exibe mensagem de empate"""
        label = myfont.render("Game tied!", True, c.BOARD_COLOR)
        self.screen.blit(label, (400, 15))
        pygame.display.update()


    def quit() -> None:
        """Encerra o jogo"""
        pygame.quit()
        sys.exit()


## generate_dataset.py


##### 

##### OBJETIVO:
O código simula partidas do jogo Connect Four entre um jogador aleatório e um jogador com IA (MCTS),
com o objetivo de gerar um dataset de jogadas e resultados, que será salvo como um arquivo CSV para uso posterior
(em IA, análise de jogadas, etc.).


##### IMPORTAÇÕES:
- numpy e pandas: usados para manipulação de arrays e criação do DataFrame.
- random: usado para gerar jogadas aleatórias e escolher quando salvar estados.
- game.board: classe Board que representa o tabuleiro.
- game.constants: constantes como peças dos jogadores.
- game.rules: regras do jogo (validação de jogadas, verificação de vitória/empate).
- ai.mcts: implementação da IA com Monte Carlo Tree Search (MCTS).


##### FUNÇÃO PRINCIPAL: `generate_dataset()`
Parâmetros:
- num_games (default=100): número de partidas simuladas.
- num_samples_per_game (default=1000): não é usado diretamente, mas pode controlar amostragem futura.

Processo:
1. Inicializa uma lista 'dataset' para armazenar dados.

2. Para cada partida (loop num_games):
    - Inicializa um novo tabuleiro (Board).
    - Define o jogador inicial como PLAYER1.
    - Enquanto o jogo não termina (game_over = False):
        
        a. Se for o turno do jogador 1 (PLAYER1):
            - Escolhe uma jogada aleatória entre as válidas.
        
        b. Se for o turno do jogador 2 (PLAYER2):
            - Cria uma árvore MCTS a partir do estado atual.
            - Executa a busca MCTS por 3 segundos para escolher a melhor jogada.
        
        c. Com 20% de chance, salva o estado do tabuleiro atual no dataset:
            - Achata o tabuleiro 2D em uma lista 1D.
            - Verifica se a jogada atual levou à vitória, empate, ou continua.
            - Se houver resultado ('win', 'loss' ou 'draw'), salva o estado + jogada + resultado.
        
        d. Aplica a jogada no tabuleiro com simulate_move().
        e. Verifica se o jogo terminou (vitória ou empate).
        f. Alterna o jogador para o próximo turno.

3. Após todas as partidas:
    - Define as colunas do DataFrame: pos_0 a pos_41 (42 posições do tabuleiro) + 'move' + 'outcome'.
    - Cria o DataFrame com pandas.
    - Salva o dataset como CSV no caminho: 'datasets/connect4_dt.csv'.

4. Se o script for executado diretamente (não importado):
    - Chama generate_dataset()
    - Exibe no console a quantidade de amostras geradas.


##### SAÍDA:
- Um arquivo CSV chamado 'connect4_dataset.csv' contendo:
    - O estado do tabuleiro.
    - A jogada executada.
    - O resultado ('win', 'loss' ou 'draw').




In [ ]:
# Importação de bibliotecas essenciais
import numpy as np
import pandas as pd
from game.board import Board                  # Classe para representar o tabuleiro
from game import constants as c               # Constantes como peças dos jogadores
from game import rules as game                # Regras do jogo (movimentos válidos, vitória, empate)
from ai.mcts import MCTS, Node                # IA com Monte Carlo Tree Search
import random

# Função para gerar um dataset simulando partidas de Connect Four
def generate_dataset(num_games=100, num_samples_per_game=1000):
    dataset = []  # Lista para armazenar os dados gerados

    for _ in range(num_games):  # Executa várias partidas
        board_obj = Board()                     # Inicializa novo tabuleiro
        board = board_obj.get_board()           # Obtém estado inicial do tabuleiro
        game_over = False                       # Flag de fim de jogo
        current_player = c.PLAYER1_PIECE        # Jogador 1 começa a partida

        while not game_over:
            if current_player == c.PLAYER1_PIECE:
                # Jogada aleatória para o Jogador 1
                available_moves = game.available_moves(board)
                move = random.choice(available_moves)
            else:
                # Jogada do Jogador 2 usando IA com MCTS
                root_node = Node(board=board, last_player=current_player)
                mcts = MCTS(root_node)
                move = mcts.search(3)  # Executa a busca por 3 segundos

            # 20% de chance de salvar este estado no dataset
            if random.random() < 0.2:
                # Converte o tabuleiro 2D em uma lista 1D
                flattened_board = [item for row in board for item in row]

                # Define o resultado da jogada se o jogo terminar aqui
                if game.winning_move(board, current_player):
                    outcome = 'win' if current_player == c.AI_PIECE else 'loss'
                elif game.is_game_tied(board):
                    outcome = 'draw'
                else:
                    outcome = None  # Não salva se o jogo ainda estiver em andamento

                if outcome:  # Só salva se houver resultado
                    dataset.append(flattened_board + [move, outcome])

            # Aplica a jogada no tabuleiro
            board = game.simulate_move(board, current_player, move)

            # Verifica fim de jogo
            if game.winning_move(board, current_player) or game.is_game_tied(board):
                game_over = True

            # Alterna o jogador
            current_player = (
                c.PLAYER2_PIECE if current_player == c.PLAYER1_PIECE else c.PLAYER1_PIECE
            )

    # Define os nomes das colunas (42 posições do tabuleiro + jogada + resultado)
    columns = [f'pos_{i}' for i in range(42)] + ['move', 'outcome']

    # Cria DataFrame com os dados
    df = pd.DataFrame(dataset, columns=columns)

    # Salva como CSV
    df.to_csv('datasets/connect4_dt.csv', index=False)

    return df

# Executa a geração do dataset se rodar o script diretamente
if __name__ == '__main__':
    print("Generating Connect Four dataset...")
    dataset = generate_dataset(num_games=100, num_samples_per_game=1000)
    print(f"Dataset generated with {len(dataset)} samples. Saved to datasets/connect4_dataset.csv")


## main.py

##### 
Este código é o ponto de entrada principal de um jogo, estruturado em uma arquitetura modular com separação entre a lógica do jogo e a interface de interação com o jogador. Resumidamente o script inicializa o tabuleiro e a interface do jogo e então inicia a execução do jogo chamando o método 'start_game'.

#### Explicação linha por linha:

1. from game.board import Board
   - Importa a classe 'Board' do módulo 'board', que está localizado dentro do pacote 'game'.
   - A classe 'Board' provavelmente representa o tabuleiro ou o estado principal do jogo.

2. from interface.interface import Interface
   - Importa a classe 'Interface' do módulo 'interface', que está localizado dentro do pacote 'interface'.
   - A classe 'Interface' provavelmente cuida da interação com o usuário (exibição e entrada de dados).

3. def main() -> None:
   - Define a função principal chamada 'main'.
   - A anotação '-> None' indica que essa função não retorna nenhum valor.

4. board = Board()
   - Cria uma instância da classe 'Board', ou seja, inicializa o tabuleiro do jogo.

5. interface = Interface()
   - Cria uma instância da classe 'Interface', ou seja, prepara a interface que será usada para interagir com o jogador.

6. interface.start_game(board)
   - Inicia o jogo chamando o método 'start_game' da interface.
   - O tabuleiro (board) é passado como argumento, indicando que a interface utilizará o estado do jogo para exibir e controlar o progresso da partida.

7. if __name__ == "__main__":
   - Verifica se o script está sendo executado diretamente (e não importado como módulo).
   - Se for o caso, chama a função 'main' para iniciar o jogo.

8. main()
   - Executa a função principal do programa, dando início à execução do jogo.




In [ ]:
from game.board import Board
from interface.interface import Interface

def main() -> None:
    board = Board()
    interface = Interface()
    interface.start_game(board)
    
if __name__ == "__main__":
    main()

## Datasets

In [ ]:
import pandas as pd

# Carrega o dataset
dfConnect4 = pd.read_csv("../src/datasets/connect4_dataset.csv")
dfIris = pd.read_csv("../src/datasets/iris.csv")

# Mostra as primeiras linhas
print(dfIris.info())
print(dfConnect4.info())
